In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams

rcParams["figure.max_open_warning"] = False

In [ ]:
def plot_performance(absolute_err, xs=None, xlabel="", ylabel="absolute error"):
    """ Plot error evolution over xs and as cdf. """
    if xs is None:
        xs = range(len(absolute_err.values()[0]))
    
    from matplotlib.lines import Line2D
    markers = [m for m in list(Line2D.markers.keys()) if m not in [".", "", ","]]
    
    i = 0
    fig, axs = plt.subplots(1, 2, squeeze=False)
    fig.set_size_inches(10, 5)
    for method, absolute_errors in absolute_err.items():
        markersize = 8 - i
        
        axs[0, 0].plot(
            xs,
            absolute_errors,
            label=method,
            marker=markers[i],
            ls=":",
            markersize=markersize,
        )
        
        xvals = sorted(absolute_errors)
        yvals = np.linspace(0, 1, len(xvals))
        axs[0, 1].plot(
            xvals, yvals, label=method, marker=markers[i], ls=":", markersize=markersize
        )
        i += 1

    # axs[0, 0].set_yscale('log')
    axs[0, 0].set_ylabel(ylabel)
    axs[0, 0].set_xlabel(xlabel)
    axs[0, 0].legend(loc="upper right")

    axs[0, 1].set_ylabel("cdf")
    axs[0, 1].set_xlabel(ylabel)
    axs[0, 1].grid(which="both")
    axs[0, 1].legend(loc="lower right")
    return fig, axs

# Experimental distance-frequency matrix

In [ ]:
from simulation import get_setup

distance = 5

fig, ax = plt.subplots()
fig.set_size_inches(5, 5)
for yaw_deg, marker in zip([0,15,30], ["o", "*", "x"]):
    source, mic_positions = get_setup(distance_cm=distance, yaw_deg=yaw_deg)
    source_image = [source[0], -source[1]]
    for i, mic in enumerate(mic_positions):
        label = f"mic{i}" if yaw_deg == 0 else None
        ax.scatter(*mic*100, label=label, marker=marker, color=f"C{i}")
    ax.scatter(*mic*100, label=f"yaw {yaw_deg}deg", marker=marker, color=f"C{i}")
ax.plot([4.8*100, 5.2*100], [0, 0], color="k", label="wall")
ax.set_title(f"setup for distance={distance}cm and different angles")

handles, labels = ax.get_legend_handles_labels()
h_mics = {l: h for h, l in zip(handles, labels) if 'mic' in l}
l1 = ax.legend(h_mics.values(), h_mics.keys(),loc="lower right", bbox_to_anchor=[.3, 0])

h_other = {l: h for h, l in zip(handles, labels) if not 'mic' in l}
ax.legend(h_other.values(), h_other.keys(),loc="lower left", bbox_to_anchor=[.3, 0])
ax.add_artist(l1)

ax.set_xlabel("x [cm]")
ax.set_ylabel("y [cm]")
ax.axis("equal")

fig.savefig(f'plots/setup.pdf', bbox_inches='tight')

In [ ]:
# exp_name = '2021_02_09_wall_tukey';
# exp_name = '2021_02_09_wall';
#exp_name = "2021_02_23_wall"
#exp_name = "2021_02_25_wall"
exp_name = "2021_03_01_flying"
# exp_name = '2021_02_19_windows';
# exp_name = '2020_12_9_rotating';
# exp_name = '2020_11_26_wall';
# fname = f'results/{exp_name}_real.pkl'
fname = f"../experiments/{exp_name}/all_data.pkl"
# fname = f'../experiments/{exp_name}/battery_data.pkl' # for 2021_02_09_wall, battery study

try:
    df_all = pd.read_pickle(fname)
    print("read", fname)
except Exception as e:
    print(e)
    print("Error: run wall_analysis.py to parse experiments.")
df_all.iloc[:, :8]

# plot_spectrograms(df_freq)

# 0. Debug parsing

In [ ]:
from bin_selection import generate_sweep
from crazyflie_description_py.parameters import N_BUFFER, FS
from dataset_parameters import kwargs_datasets
from pandas_utils import filter_by_dicts
from plotting_tools import *
import seaborn as sns
from wall_detector import WallDetector, normalized_std
from frequency_analysis import get_index_matrix

mag_thresh = 1.0 # tested for 25_* and 23_* dataset
ymin = 1e-4
ymax = 3e2
degree = 0
if len(df_all.distance.unique()) > 1:
    distance = 20
    chosen_dict = {
        "degree": degree, 
        "distance": distance, 
    }
else:
    distance = df_all.iloc[0].distance
    print('using unique distance', distance)
    chosen_dict = {
        "degree": degree, 
    }

#chosen_dict["motors"] = "all45000"
#chosen_dict["mic_type"] = "measurement"
#chosen_dict["mic_type"] = "audio_deck"

df_filtered = filter_by_dicts(df_all, [chosen_dict])
print("plotting\n", df_filtered.iloc[:, :8])

filter_by_motors = filter_by_mic_type = filter_by_appendix = False
if len(df_filtered.motors.unique()) > 1:
    filter_by_motors = True
if len(df_filtered.mic_type.unique()) > 1:
    filter_by_mic_type = True
if len(df_filtered.appendix.unique()) > 1:
    filter_by_appendix = True

verbose = False

fig_slice, ax_slice = plt.subplots()
fig_slice.set_size_inches(5, 5)
    
for mic_type, df_mic_types in df_filtered.groupby("mic_type"):
    if mic_type == "measurement":
        mic = 0
    else:
        mic = 1
        
    for j, (i_row, row) in enumerate(df_mic_types.iterrows()):
        if (mic_type == "measurement") and (row.snr == 1):  # this is no "real snr"
            continue

            
        label = title = ""
        if filter_by_mic_type:
            label += f"{mic_type}, "
        if filter_by_motors:
            label += f"motors={row.motors}"
        if filter_by_appendix:
            label += f"{row.appendix.replace('_', ' ')} "
        label += f"mic{mic}"

        # processing
        wall_detector = WallDetector(exp_name=exp_name, mic_type=row.mic_type)
        try:
            spec_masked_all, freqs_masked = wall_detector.fill_from_row(row)
        except:
            print("skipping", row)
            continue
        wall_detector.remove_bad_freqs(mag_thresh=mag_thresh, verbose=verbose, dryrun=False)
        wall_detector.merge_close_freqs(verbose=verbose)
        wall_detector.remove_spurious_freqs(verbose=verbose)

        # plotting
        index_matrix = get_index_matrix(spec_masked_all)
        spec_masked = np.mean(spec_masked_all, axis=1)
        
        fig, ax = plt.subplots()
        fig.set_size_inches(5, 5)
        im = ax.pcolorfast(row.seconds, freqs_masked, np.log10(spec_masked[:-1, :-1]))
        add_colorbar(fig, ax, im)
        ax.scatter(row.seconds[1:], 
                   freqs_masked[index_matrix[0, :]][:-1], 
                   color='C1', ls=':', label='highest frequency', s=5)
        ax.legend()
        ax.set_xlim(
            kwargs_datasets[exp_name][row.mic_type]["min_time"],
            kwargs_datasets[exp_name][row.mic_type]["max_time"],
        )
        ax.set_ylim(
            kwargs_datasets[exp_name][row.mic_type]["min_freq"],
            kwargs_datasets[exp_name][row.mic_type]["max_freq"],
        )
        ax.set_xlabel("time [s]")
        ax.set_ylabel("frequency [Hz]")
        ax.set_title(label)
        save_fig(
            fig, f"plots/{exp_name}_{row.mic_type}_{row.motors}{row.appendix}_spec.png"
        )

        df_mic = wall_detector.df[wall_detector.df.mic == mic]
        
        stds = df_mic.groupby("frequency", sort=True).magnitude.apply(normalized_std).values
        #stds = df_mic.groupby("frequency", sort=True).magnitude.apply(np.nanstd).values
        medians = df_mic.groupby("frequency", sort=True).magnitude.apply(np.nanmedian).values
        stds = np.r_[[np.zeros(len(medians))], [2*stds]]
        ax_slice.errorbar(x=np.sort(df_mic.frequency.unique()), 
                          y=medians, 
                          yerr=stds, 
                          label=label, marker='o', markersize=4.0, lw=2.0)
        #sns.scatterplot(
        #    data=df_mic, x="frequency", y="magnitude", hue="counter", ax=ax, linewidth=0
        #)

        fig_raw, ax = plot_raw_signals(spec_masked_all, freqs_masked, mic_idx=mic)
        ax.set_ylim(ymin, ymax)
        ax.set_title(label)
        save_fig(
            fig_raw, f"plots/{exp_name}_{row.mic_type}_{row.motors}{row.appendix}_raw.png"
        )
        
        normalized_stds = df_mic.groupby("frequency", sort=True)["magnitude"].apply(normalized_std).values
        frequencies = np.sort(df_mic.frequency.unique())
        fig_std, ax = plt.subplots()
        ax.plot(frequencies, normalized_stds, marker='o')
        ax.set_title(label)
        
ax_slice.set_ylabel('amplitude')
ax_slice.set_xlabel("frequency [Hz]")
ax_slice.set_yscale("log")
ax_slice.set_ylim(1e-1, 1e2)
ax_slice.grid(which="both")

title = f'distance={row.distance}cm mic{mic}'
ax_slice.set_title(title)
#ax_slice.legend(loc="upper left", bbox_to_anchor=[1.0, 1.0])
ax_slice.legend()
save_fig(fig_slice, f"plots/{exp_name}{row.appendix}_slice.png")

# Full analysis

## 1. Regenerate or read results

In [ ]:
from wall_analysis import get_df_matrices, FILTERS

#exp_name = "2021_02_25_wall"
exp_name = "2021_02_23_wall"

fname = "results/wall_analysis.pkl"
# method = 'dryrun'
# method = 'new'
#method = 'replace'
method = "read"

if method == "read":
    results_df = pd.read_pickle(fname)
elif method == "replace":
    from pandas_utils import fill_df

    FILTERS = ["mic_type", "snr", "motors"]
    results_df = pd.read_pickle(fname)

    print("read", fname)
    results_df_new = get_df_matrices(exp_name)

    for chosen_tuple, df in results_df_new.groupby(FILTERS):
        assert len(df) == 1
        row = df.iloc[0]

        filter_dict = dict(zip(FILTERS, chosen_tuple))
        filter_dict.update({"exp_name": exp_name})
        fill_dict = {
            "df_dist": row.df_dist,
            "df_freq": row.df_freq,
            "df_matrix": row.df_matrix,
        }
        print("filling results_df_old...")
        results_df = fill_df(results_df, filter_dict, fill_dict)
    pd.to_pickle(results_df, fname)
    print("saved as", fname)
elif method == "new":
    results_df = get_df_matrices(exp_name)
    pd.to_pickle(results_df, fname)
    print("saved as", fname)
elif method == "dryrun":
    results_df = get_df_matrices(exp_name, max_distance=10, plot=True)
results_df

## 2. Pick dataset

In [ ]:
from pandas_utils import filter_by_dicts

#exp_name = "2021_02_23_wall"
exp_name = "2021_02_25_wall"

# mic_type = 'measurement'
mic_type = "audio_deck"
motors = 'all45000'
#motors = 0 

fname = f"{exp_name}_{mic_type}_{motors}"

filter_dict = dict(exp_name=exp_name, motors=motors, mic_type=mic_type)

df = filter_by_dicts(results_df, [filter_dict])
assert len(df) == 1
row = df.iloc[0]
row

In [ ]:
from calibration import get_calibration_function, get_calibration_function_dict
from wall_detector import normalize_df_matrix
from copy import deepcopy
from simulation import get_df_theory

fig, ax = plt.subplots()
calib_function_old = get_calibration_function(ax=ax)
ax.set_title("old calibration")
fig, ax = plt.subplots()
calib_function = get_calibration_function_dict(**filter_dict, ax=ax)
ax.set_title("new calibration")

max_distance = 50
#distances_grid = np.arange(0, max_distance + 1)
distances_grid = np.linspace(0, max_distance, 100)

if row.mic_type == "measurement":
    df_theory = get_df_theory(row.df_freq, distances_grid, chosen_mics=[1])
else:
    df_theory = get_df_theory(row.df_freq, distances_grid, chosen_mics=range(4))

In [ ]:
from wall_detector import prune_df_matrix

df_matrix, df_freq, indices = prune_df_matrix(
    row.df_matrix, row.df_freq, ratio_missing_allowed=0.5, verbose=True
)
df_theory_pruned = df_theory[:, indices, :]

# 1. study distance slices

In [ ]:
def fill_nans(slice_f, xs):
    for m in range(slice_f.shape[0]):
        mask = np.isfinite(slice_f[m])
        slice_f[m, :] = np.interp(xs, xs[mask], slice_f[m, mask])
    return slice_f

In [ ]:
from constants import SPEED_OF_SOUND
def plot_distance_slice(slice_exp, slice_the, dist_exp, dist_the, plot_d=False, plot_f=True):
    
    fig = fig_f = None
    
    if plot_d:
        fig, axs = plt.subplots(1, slice_exp.shape[0], squeeze=False, sharey=True)
        fig.set_size_inches(10, 5)
        fig.suptitle(f"standardized distance slices, at frequency {f:.0f}Hz")
        
    if plot_f:
        fig_f, axs_f = plt.subplots(1, slice_exp.shape[0], squeeze=False, sharey=True)
        fig_f.set_size_inches(10, 5)
        fig_f.suptitle(f"FFT of standardized distance slices, at frequency {f:.0f}Hz")

    for m in range(slice_exp.shape[0]):
        slice_exp_norm = deepcopy(slice_exp[m])
        slice_the_norm = deepcopy(slice_the[m])
        
        
        slice_exp_norm -= np.mean(slice_exp_norm)
        slice_exp_norm /= np.max(slice_exp_norm) - np.min(slice_exp_norm)

        slice_the_norm -= np.mean(slice_the_norm)
        slice_the_norm /= np.max(slice_the_norm) - np.min(slice_the_norm)

        if plot_d:
            axs[0, m].plot(dist_the, slice_the_norm, label='theoretical')
            axs[0, m].plot(dist_exp, slice_exp_norm, label='measured')
            axs[0, m].set_title(f"mic{m}")
            axs[0, m].legend(loc='upper right')
            
        n = max(len(slice_exp_norm), 1000)
        freqs_exp = np.fft.rfftfreq(n, d=dist_exp[1]-dist_exp[0]) # unit: 1/cm
        freqs_theory = np.fft.rfftfreq(n, d=dist_the[1]-dist_the[0]) # unit: 1/cm
        
        fft_exp = np.abs(np.fft.rfft(slice_exp_norm, n=n))
        fft_exp /= np.sum(fft_exp)

        fft_theory = np.abs(np.fft.rfft(slice_the_norm, n=n))
        fft_theory /= np.sum(fft_theory)

        if plot_f:
            axs_f[0, m].plot(freqs_theory, fft_theory, label='theoretical')
            axs_f[0, m].plot(freqs_exp, fft_exp, label='measured')
            axs_f[0, m].set_title(f"mic{m}")
            axs_f[0, m].legend(loc='upper right')
    return fig, fig_f


valid_freqs = df_freq[~np.any(np.isnan(df_matrix), axis=(0, 2))]
plot_freqs = [valid_freqs[0], valid_freqs[len(valid_freqs)//2], valid_freqs[-2]]
for i, f in enumerate(df_freq):
    slice_exp=df_matrix[:, i, :]
    if np.any(np.isnan(slice_exp)):
        slice_exp = fill_nans(slice_exp, row.df_dist)
    
    fig, fig_f = plot_distance_slice(
        slice_exp=slice_exp,
        slice_the=df_theory_pruned[:, i, :],
        dist_exp=row.df_dist,
        dist_the=distances_grid,
        plot_d=True, 
        plot_f=True,
    )
    if f in plot_freqs:
        save_fig(fig, f'plots/{fname}_distance_slice_{f:.0f}.pdf')

In [ ]:
# distance slice algorithm
gt = 90
abs_err_dict = {f"mic{m}": [np.nan]*len(df_freq) for m in range(df_matrix.shape[0])}

for i, f in enumerate(df_freq):
    thetas = []
    
    slice_exp = df_matrix[:, i, :]
    for m in range(slice_exp.shape[0]):
        slice_exp_norm = deepcopy(slice_exp[m])
        slice_exp_norm -= np.mean(slice_exp_norm)
        n = max(len(slice_exp_norm), 1000)
        
        freqs_exp = np.fft.rfftfreq(n, d=row.df_dist[1]-row.df_dist[0]) # unit: 1/cm
        fft_exp = np.abs(np.fft.rfft(slice_exp_norm, n=n))
        
        if np.argmax(fft_exp) == 0:
            continue

        period_exp = 1 / freqs_exp[np.argmax(fft_exp)] # unit: cm
        period_theory = SPEED_OF_SOUND / 2 / f * 100 # unit: cm
        ratio = period_exp / period_theory
        if ratio > 1:
            print('Warning for ratio bigger than 1:', ratio)
            ratio = 1
        #ratio = min(period_exp / period_theory, 1)
        theta = 180 / np.pi * np.arcsin(ratio)
        abs_err_dict[f"mic{m}"][i] = np.abs(theta - gt)

In [ ]:
fig, axs = plot_performance(abs_err_dict, xs=df_freq, xlabel="frequency [Hz]", ylabel="absolute error [deg]")
save_fig(fig, f'plots/{fname}_distance_slice_performance.pdf')

# 2. study frequency slices

In [ ]:
min_value = np.inf
max_value = -np.inf

distances_idx = np.argmin(np.abs(distances_grid[:, None] - row.df_dist[None, :]), axis=0)

results = pd.DataFrame(columns=["normalization", "matrix", "values"])
method_dict = {
    'raw': "",
    'normalize': "normalize",
    'calibration-offline-old': calib_function_old,
    'calibration-offline': calib_function,
    'calibration-online': "calibration-online",
    'standardize': "standardize",
    'zero_mean': "zero_mean",
    'theoretical': "theoretical",
}

if row.mic_type == "measurement":
    plot_channel_idx = 0  # only choice
else:
    plot_channel_idx = 1  # corresponds to mic below measurement mic

__, df_freq, indices = prune_df_matrix(row.df_matrix, row.df_freq)
df_theory_pruned = df_theory[:, indices, :]

for j, (key, method) in enumerate(method_dict.items()):
    values = None
    df_matrix = row.df_matrix[:, indices, :]

    if key == "raw":
        df_norm = deepcopy(df_matrix)
    elif key == "theoretical":
        df_norm = deepcopy(df_theory_pruned[:, :, distances_idx])
    else:
        df_norm, values = normalize_df_matrix(
            df_matrix=df_matrix, freqs=df_freq, method=method
        )
    results.loc[len(results), :] = {
        "normalization": key,
        "matrix": df_norm,
        "values": values,
    }
    # print(key, data_type, df_norm.shape)
    min_value = min(min_value, -abs(np.min(df_norm)))
    max_value = max(max_value, abs(np.max(df_norm)))

In [ ]:
from calibration import get_calibration_function_matrix

fig, ax = plt.subplots()
get_calibration_function_matrix(row.df_matrix, row.df_freq, ax=ax)

In [ ]:
min_freq = min(df_freq)
max_freq = max(df_freq)

df_the = df_theory_pruned[plot_channel_idx, :, distances_idx].T
min_value = None
max_value = None

for normalization, df in results.groupby("normalization", sort=False):
    df_exp = df.iloc[0].matrix[plot_channel_idx]

    fig, ax = plt.subplots()
    fig.set_size_inches(5, 5)
    ax, im = plot_df_matrix(
        row.df_dist,
        df_freq,
        df_exp,
        ax=ax,
        min_freq=min_freq,
        max_freq=max_freq,
        vmin=min_value,
        vmax=max_value,
    )
    add_colorbar(fig, ax, im)
    ax.set_title(f"{normalization}")
    ax.set_xlabel('distance [cm]')
    ax.set_ylabel('frequency [Hz]')
    
    save_fig(fig, f"plots/{fname}_matrices_{normalization}.png")

# 3. Visualize cost function

In [ ]:
from simulation import get_freq_slice_theory
from wall_detector import get_probability_cost, get_probability_fft

mic_idx = 1

# chosen_methods = results.normalization.unique()
chosen_methods = ["raw", "calibration-online", "standardize", "theoretical"]

# TODO(FD) actually choose good frequencies here
chosen_frequencies = df_freq

freq_indices = np.where(chosen_frequencies[None, :] == df_freq[:, None])[0]
np.testing.assert_allclose(df_freq[freq_indices], chosen_frequencies)

absolute_err_dict = {
    key: {method: [] for method in chosen_methods} for key in ["cost", "fft"]
}

distances = row.df_dist[row.df_dist <= max_distance]

plot_distances = [2, 5, 10]

for i_d, distance in enumerate(distances):

    fig, ax = plt.subplots()
    fig.set_size_inches(5, 5)
    fig_cost, ax_cost = plt.subplots()
    fig_cost.set_size_inches(5, 5)
    fig_fft, ax_fft = plt.subplots()
    fig_fft.set_size_inches(5, 5)
    
    for i_method, method in enumerate(chosen_methods):
        df = results.loc[results.normalization == method]
        slice_exp = df.iloc[0].matrix[[mic_idx], freq_indices, i_d]

        # doing this here for plotting reasons. Doesn't change performance as
        # this is done again in get_probability_cost
        slice_exp -= np.mean(slice_exp)
        slice_exp /= np.std(slice_exp)

        proba_cost = get_probability_cost(
            slice_exp, df_freq[freq_indices], 
            distances_grid, mic_idx=mic_idx
        )
        distances_fft, proba_fft = get_probability_fft(
            slice_exp, 
            df_freq[freq_indices]
        )

        d_cost_idx = np.argmax(proba_cost)
        d_cost = distances_grid[d_cost_idx]
        abs_err = abs(d_cost - distance)
        absolute_err_dict["cost"][method].append(abs_err)

        ax_cost.semilogy(distances_grid, proba_cost, color=f"C{i_method}", label=method)
        ax_cost.axvline(x=d_cost, color=f"C{i_method}")
        ax_cost.axvline(x=distance, color="black", ls=":")
        ax_cost.set_xlabel("distances [cm]")
        ax_cost.set_ylabel("probability")
        ax_cost.set_xlim(min(distances_grid), max(distances_grid))
        ax_cost.set_ylim(1e-5, 1)
        ax_cost.set_title(f"probability for distance {distance:.0f}cm")
        
        d_fft_idx = np.argmax(proba_fft)
        d_fft = distances_fft[d_fft_idx]
        absolute_err_dict["fft"][method].append(abs(d_fft - distance))
        

        ax_fft.semilogy(distances_fft, proba_fft, color=f"C{i_method}", ls=":")
        ax_fft.axvline(x=d_fft, color=f"C{i_method}", ls=":")
        ax_fft.axvline(x=distance, color="black", ls=":")
        ax_fft.set_xlim(min(distances_grid), max(distances_grid))
        ax_fft.set_ylim(1e-5, 1)
        ax_fft.set_xlabel("distances [cm]")
        ax_fft.set_ylabel("probability")
        ax_fft.set_title(f"probability for distance {distance:.0f}cm")

        # plotting
        ax.plot(chosen_frequencies, slice_exp, label=method, color=f"C{i_method}")
        ax.set_xlim(min_freq, max_freq)
        ax.set_ylim(-3, 3)
        ax.set_xlabel("frequency [Hz]")
        ax.set_ylabel("amplitude")
        ax.set_title(f"slices for distance {distance:.0f}cm")
        

    ax.legend(loc="lower right")
    ax_cost.legend(loc="lower right")

    if distance in plot_distances:
        fname_here = f'plots/{fname}_{distance:.0f}_slice.png'
        save_fig(fig, fname_here)
        
        fname_here = f'plots/{fname}_{distance:.0f}_cost.png'
        save_fig(fig_cost, fname_here)

# 4. Evaluate algorithm performance

In [ ]:
for key, absolute_err in absolute_err_dict.items():
    fig, axs = plot_performance(absolute_err, xs=distances, xlabel="distance [cm]", ylabel="absolute error [cm]")
    axs[0, 1].set_xlim(-1, max(distances))
    # fname = f'results/{exp_name}/performance_{mic_type}_snr{snr}_subset.png'
    # fname = f'results/{exp_name}/performance_{mic_type}_snr{snr}_all.png'
    fname_here = f"plots/{fname}_{key}_frequency_performance.png"
    save_fig(fig, fname_here)

In [ ]:
# old stuff

In [ ]:
from scipy.interpolate import UnivariateSpline, LSQUnivariateSpline
from numpy.polynomial import Chebyshev, Legendre, Polynomial, Laguerre

spline_k = 2
spline_ext = 3

for mic_idx in range(row.df_matrix.shape[0]):
    df_norm, values = normalize_df_matrix(
        df_matrix=row.df_matrix, freqs=row.df_freq, method="calibration-online"
    )

    df_norm, df_freq, indices = prune_df_matrix(df_norm, row.df_freq)
    values = values[:, indices]

    xvalues = df_freq
    yvalues = np.log10(values[mic_idx, :, 0])

    plt.figure()
    plt.plot(xvalues, yvalues, label="raw", color="C0", marker="*")
    for i, poly in enumerate([Legendre]):  
        c = poly.fit(xvalues, yvalues, deg=10)
        x, y = c.linspace()
        plt.plot(x, y, label=poly.__name__, color=f"C{i+1}")

    # spline = UnivariateSpline(xvalues, yvalues, k=2)
    # plt.plot(xvalues, spline(xvalues), label='Spline', color='C5')

    knots = np.linspace(min(df_freq), max(df_freq), 7)[1:-1]
    spline = LSQUnivariateSpline(xvalues, yvalues, t=knots, k=spline_k, ext=spline_ext)
    plt.plot(xvalues, spline(xvalues), label="LSQSpline", color=f"C{i+2}", marker="o")
    [plt.axvline(k, color=f"C{i+2}") for k in knots]
    plt.legend()
    plt.title(f"mic{mic_idx}")

In [ ]:
def spline_interpolation(xvalues, df_matrix):
    """ Generate spline interpolation for all distances.
    
    :return calib_values: mics x freqs x distances, interpolated spline curve
    """
    knots = np.linspace(min(df_freq), max(df_freq), 6)[1:-1]
    calib_values = np.empty_like(df_matrix)
    for i, dist in enumerate(row.df_dist):
        for m in range(df_matrix.shape[0]):

            slice_f = np.log10(df_matrix[m, :, i])
            mask = ~np.isnan(slice_f)

            spline = LSQUnivariateSpline(
                xvalues[mask], slice_f[mask], t=knots, k=spline_k, ext=spline_ext
            )
            calib_values[m, :, i] = 10 ** spline(xvalues)
    return calib_values


df_matrix, df_freq, __ = prune_df_matrix(row.df_matrix, row.df_freq)
calib_values = spline_interpolation(df_freq, df_matrix)

fig, axs = plt.subplots(1, calib_values.shape[0], squeeze=False, sharey=True)
fig.set_size_inches(10, 5)
cmap = plt.get_cmap("inferno")
for m in range(calib_values.shape[0]):
    for i, dist in enumerate(row.df_dist):
        axs[0, m].semilogy(
            df_freq, calib_values[m, :, i], color=cmap(i / len(row.df_dist))
        )
    axs[0, m].semilogy(
        df_freq, calib_values[m, :, 0], color=cmap(0), label=f"{row.df_dist[0]}cm"
    )
    axs[0, m].semilogy(
        df_freq,
        calib_values[m, :, i],
        color=cmap(i / len(row.df_dist)),
        label=f"{row.df_dist[-1]}cm",
    )
    axs[0, m].set_title(f"mic{m}")
    axs[0, m].legend(loc="lower right")